# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import GAPI_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [9]:
# Import the WeatherPy_database.csv file. 
weather_df = pd.read_csv("Weather.csv")

# Display sample data
weather_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,80.76,74,40,6.91,scattered clouds
1,1,Tabou,CI,4.4230,-7.3528,79.45,87,98,8.01,moderate rain
2,2,Luau,AO,-10.7073,22.2247,65.37,93,100,3.13,overcast clouds
3,3,Turangi,NZ,-39.0000,175.9333,59.07,43,1,2.15,clear sky
4,4,Ushuaia,AR,-54.8000,-68.3000,53.26,47,40,4.61,light rain


In [16]:
weather_df.count()

City_ID                708
City                   708
Country                702
Lat                    708
Lng                    708
Max Temp               708
Humidity               708
Cloudiness             708
Wind Speed             708
Current Description    708
dtype: int64

## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [ ]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your vacation ?"))
max_temp = float(input("What is the maximum temperature you would like for  your vacation?"))

In [22]:
city_data_df = weather_df.loc[(weather_df["Max Temp"]>=70) & (weather_df["Max Temp"]<=90)]
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,80.76,74,40,6.91,scattered clouds
1,1,Tabou,CI,4.4230,-7.3528,79.45,87,98,8.01,moderate rain
11,11,Dingle,PH,10.9995,122.6711,72.61,91,2,8.12,clear sky
12,12,Huarmey,PE,-10.0681,-78.1522,77.88,70,99,9.13,overcast clouds
13,13,Vila Do Maio,CV,15.1333,-23.2167,75.99,75,77,10.92,broken clouds


In [23]:
city_data_df.count()

City_ID                261
City                   261
Country                261
Lat                    261
Lng                    261
Max Temp               261
Humidity               261
Cloudiness             261
Wind Speed             261
Current Description    261
dtype: int64

## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [24]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = city_data_df.dropna()


# Display sample data
clean_travel_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,80.76,74,40,6.91,scattered clouds
1,1,Tabou,CI,4.4230,-7.3528,79.45,87,98,8.01,moderate rain
11,11,Dingle,PH,10.9995,122.6711,72.61,91,2,8.12,clear sky
12,12,Huarmey,PE,-10.0681,-78.1522,77.88,70,99,9.13,overcast clouds
13,13,Vila Do Maio,CV,15.1333,-23.2167,75.99,75,77,10.92,broken clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [25]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country","Max Temp","Current Description","Lat", "Lng"]].copy()
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng
0,Vaini,TO,80.76,scattered clouds,-21.2000,-175.2000
1,Tabou,CI,79.45,moderate rain,4.4230,-7.3528
11,Dingle,PH,72.61,clear sky,10.9995,122.6711
12,Huarmey,PE,77.88,overcast clouds,-10.0681,-78.1522
13,Vila Do Maio,CV,75.99,broken clouds,15.1333,-23.2167


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [26]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] =""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,80.76,scattered clouds,-21.2000,-175.2000,
1,Tabou,CI,79.45,moderate rain,4.4230,-7.3528,
11,Dingle,PH,72.61,clear sky,10.9995,122.6711,
12,Huarmey,PE,77.88,overcast clouds,-10.0681,-78.1522,
13,Vila Do Maio,CV,75.99,broken clouds,15.1333,-23.2167,
18,Saint-Philippe,RE,80.19,overcast clouds,-21.3585,55.7679,
23,Bilma,NE,83.73,clear sky,18.6853,12.9164,
25,Atuona,PF,80.15,few clouds,-9.8000,-139.0333,
28,Touros,BR,80.26,overcast clouds,-5.1989,-35.4608,
30,Namibe,AO,71.58,few clouds,-15.1961,12.1522,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [8]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": GAPI_key,
    "limit":20,
    "filter":f"circle:{longitude},{latitude},{radius}"
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [27]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": GAPI_key,
    "limit":20,
    "filter":"",
    "bias":""
}
# Iterate through the hotel_df DataFrame

for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Vaini - nearest hotel: Keleti International Resort
Tabou - nearest hotel: hôtel le rochet
Dingle - nearest hotel: No hotel found
Huarmey - nearest hotel: Hostal Santa Rosa
Vila Do Maio - nearest hotel: Residencial Jardins De Maio
Saint-Philippe - nearest hotel: Le Baril
Bilma - nearest hotel: No hotel found
Atuona - nearest hotel: Pearl Resort
Touros - nearest hotel: Pousada Atlântico
Namibe - nearest hotel: Hotel Chik Chik Namibe
Belmonte - nearest hotel: No hotel found
Rikitea - nearest hotel: Chez Bianca & Benoit
Liberty - nearest hotel: No hotel found
Kapaa - nearest hotel: Pono Kai Resort
Hilo - nearest hotel: Dolphin Bay Hotel
Butaritari - nearest hotel: No hotel found
Avarua - nearest hotel: Paradise Inn
Ellisras - nearest hotel: GPS Hotel
Aripuana - nearest hotel: No hotel found
Vila Velha - nearest hotel: Hotel Prainha
Auki - nearest hotel: No hotel found
Mahebourg - nearest hotel: Grand Bel Air
East London - nearest hotel: No hotel found
Port Blair - nea

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,80.76,scattered clouds,-21.2000,-175.2000,Keleti International Resort
1,Tabou,CI,79.45,moderate rain,4.4230,-7.3528,hôtel le rochet
11,Dingle,PH,72.61,clear sky,10.9995,122.6711,No hotel found
12,Huarmey,PE,77.88,overcast clouds,-10.0681,-78.1522,Hostal Santa Rosa
13,Vila Do Maio,CV,75.99,broken clouds,15.1333,-23.2167,Residencial Jardins De Maio
...,...,...,...,...,...,...,...
673,Basoko,CD,78.24,overcast clouds,1.2391,23.6160,No hotel found
677,Kieta,PG,80.46,light rain,-6.2167,155.6333,No hotel found
691,Imbituba,BR,76.06,scattered clouds,-28.2400,-48.6703,Silvestre Praia Hotel
699,Sal Rei,CV,70.52,clear sky,16.1833,-22.9167,Hotel Marine Club


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [42]:
clean_hotel_df=hotel_df.loc[hotel_df["Hotel Name"]!= "No hotel found"]
clean_hotel_df.head()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,80.76,scattered clouds,-21.2000,-175.2000,Keleti International Resort
1,Tabou,CI,79.45,moderate rain,4.4230,-7.3528,hôtel le rochet
12,Huarmey,PE,77.88,overcast clouds,-10.0681,-78.1522,Hostal Santa Rosa
13,Vila Do Maio,CV,75.99,broken clouds,15.1333,-23.2167,Residencial Jardins De Maio
18,Saint-Philippe,RE,80.19,overcast clouds,-21.3585,55.7679,Le Baril


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [43]:
# Create the output File (CSV)
output_file="WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_file,index_label="City ID")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [47]:
map_plot = clean_hotel_df.hvplot.points("Lng","Lat",
                                        geo=True, tiles="OSM", frame_width= 700,
                                          frame_height = 500, size = "Max Temp", scale = 0.5, color = "City",hover_cols = ["Country","Current Description"])
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Country,Current Description)